In [1]:
import json
import pandas as pd
import numpy as np

Sensor_readings = pd.read_json('data/json/W512.w512_readings (1).json')
Aircon_Data = pd.read_json('data/json/W512.w512_aircon_status (1).json')
Weather_readings = pd.read_json('data/json/user.weather_data (1).json')

def convert_AirconData(data):
    records = []
    
    for index, row in data.iterrows():
        # Parse FC_FullStatus_Readings if it's a string representation of a dictionary
        if isinstance(row['FC_FullStatus_Readings'], str):
            fc_readings = ast.literal_eval(row['FC_FullStatus_Readings'])
        else:
            fc_readings = row['FC_FullStatus_Readings']


        try:
            combined_datetime = pd.to_datetime(f"{row['date']} {row['time']}")
            formatted_datetime = pd.to_datetime(combined_datetime.strftime("%Y-%m-%d %H:%M:%S"))

        except Exception as e:
            print(f"Error combining datetime for row {index}: {e}")
            combined_datetime = None
            formatted_datetime = None

        
        # Create a record with base information
        record = {
            'Datetime': formatted_datetime
        }
        
        # Add each FC Unit's details as separate columns
        for unit, unit_data in fc_readings.items():
            record[f'{unit}_Status'] = unit_data['Status']
            record[f'{unit}_Fan_Status'] = unit_data['Fan_Status']
            record[f'{unit}_Set_Point'] = unit_data['Set_Point']
            record[f'{unit}_Operation_Mode'] = unit_data['Operation_Mode']
        
        records.append(record)
    
    # Create DataFrame
    df = pd.DataFrame(records)
    return df


def convert_sensorReadings(data):
    records = []
    
    # List of keys to exclude from Lorawan_Readings
    include_keys_1 = ["24E124725E285123", "24E124725E331695","24E124725E331744",
                      "24E124725E332483","24E124725E290348","24E124725E331733","24E124725E286745"]#"24E124136D316361" is suppiosed to be outdoor but it is not outdoor yet
    include_keys_2 = ["Sensor_1","Sensor_3","Sensor_6"]
    
    for index, row in data.iterrows():
        # Parse Energy_Readings if it's a string representation of a dictionary
        if isinstance(row['Energy_Readings'], str):
            Energy_readings = ast.literal_eval(row['Energy_Readings'])
        else:
            Energy_readings = row['Energy_Readings']
            
        # Parse Lorawan_Readings if it's a string representation of a dictionary
        if isinstance(row['Lorawan_Readings'], str):
            Lorawan_Readings = ast.literal_eval(row['Lorawan_Readings'])
        else:
            Lorawan_Readings = row['Lorawan_Readings']

        try:
            # Combine the date and time columns to create a datetime object
            combined_datetime = pd.to_datetime(f"{row['date']} {row['time']}")
            formatted_datetime = pd.to_datetime(combined_datetime.strftime("%Y-%m-%d %H:%M:%S"))
        except Exception as e:
            print(f"Error combining datetime for row {index}: {e}")
            formatted_datetime = None

        # Create a record with base information
        record = {
            'Datetime': formatted_datetime
        }
        
        # Add each Energy sensor's details as separate columns
        for unit, unit_data in Energy_readings.items():
            if unit not in include_keys_2:
                continue
                
            record[f'{unit}_Current'] = unit_data['Current']
            record[f'{unit}_Energy'] = unit_data['Energy']
            record[f'{unit}_Power'] = unit_data['Power']
        
        # Add each Lorawan device's details as separate columns
        for unit, unit_data in Lorawan_Readings.items():
            if unit not in include_keys_1:
                continue
            record[f'{unit}_Humidity'] = unit_data.get('humidity', None)
            record[f'{unit}_Temperature'] = unit_data.get('temperature', None)

            co2_value = unit_data.get('co2', None)
            if co2_value is not None:
                record[f'{unit}_CO2'] = co2_value

        # Append the record to the list of records
        records.append(record)
    df=pd.DataFrame(records)
    return df


def convert_weatherData(data):
    records = []
    for index, row in data.iterrows():
        # Parse Energy_Readings if it's a string representation of a dictionary
        if isinstance(row['result'], str):
            weather_results = ast.literal_eval(row['result'])
        else:
            weather_results = row['result']
            
        try:
            combined_datetime = pd.to_datetime(f"{row['date']} {row['time']}")
            formatted_datetime = pd.to_datetime(combined_datetime.strftime("%Y-%m-%d %H:%M:%S"))
        except Exception as e:
            print(f"Error combining datetime for row {index}: {e}")
            formatted_datetime = None

        record = {
            'Datetime': formatted_datetime
        }  

        record['weather_status'] = weather_results['weather_status']
        record['weather_temp'] = weather_results['weather_temp']
        record['weather_humid'] = weather_results['weather_humidity']
            
        records.append(record)
    df=pd.DataFrame(records)
    return df



Aircon_data_df = convert_AirconData(Aircon_Data)
Aircon_data_df = Aircon_data_df[3194:]
Sensor_readings_df = convert_sensorReadings(Sensor_readings)
Sensor_readings_df = Sensor_readings_df.interpolate(method='linear')
weather_readings_df = convert_weatherData(Weather_readings)

# Merge Aircon data with Sensor readings using merge_asof
merged_df = pd.merge_asof(Aircon_data_df, Sensor_readings_df, on='Datetime', direction='nearest')

# Now, merge the Weather readings with the previous result using merge_asof
merged_df = pd.merge_asof(merged_df, weather_readings_df, on='Datetime', direction='nearest')


merged_df['total_energy'] = (
    merged_df['Sensor_1_Energy'] +
    merged_df['Sensor_3_Energy'] +
    merged_df['Sensor_6_Energy']
)

merged_df['total_power'] = (
    merged_df['Sensor_1_Power'] +
    merged_df['Sensor_3_Power'] +
    merged_df['Sensor_6_Power']
)

merged_df['total_current'] = (
    merged_df['Sensor_1_Current'] +
    merged_df['Sensor_3_Current'] +
    merged_df['Sensor_6_Current']
)

temperature_col = [
    col for col in merged_df.columns 
    if "24e124" in col.lower() and "temperature" in col.lower()
]
humidity_col = [
    col for col in merged_df.columns 
    if "24e124" in col.lower() and "humidity" in col.lower()
]
co2_col = [
    col for col in merged_df.columns 
    if "24e124" in col.lower() and "co2" in col.lower()
]

merged_df['avg_temperature'] = merged_df[temperature_col].mean(axis=1)
merged_df['avg_humidity'] = merged_df[humidity_col].mean(axis=1)
merged_df['avg_co2'] = merged_df[co2_col].mean(axis=1)

merged_df['hour'] = pd.to_datetime(merged_df['Datetime']).dt.hour
merged_df['Day_of_week'] = pd.to_datetime(merged_df['Datetime']).dt.dayofweek

merged_df.info()

merged_df.to_csv('data.csv')



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2438 entries, 0 to 2437
Data columns (total 69 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   Datetime                      2438 non-null   datetime64[ns]
 1   FC_Unit_1_Status              2438 non-null   object        
 2   FC_Unit_1_Fan_Status          2438 non-null   object        
 3   FC_Unit_1_Set_Point           2438 non-null   float64       
 4   FC_Unit_1_Operation_Mode      2438 non-null   object        
 5   FC_Unit_2_Status              2438 non-null   object        
 6   FC_Unit_2_Fan_Status          2438 non-null   object        
 7   FC_Unit_2_Set_Point           2438 non-null   float64       
 8   FC_Unit_2_Operation_Mode      2438 non-null   object        
 9   FC_Unit_3_Status              2438 non-null   object        
 10  FC_Unit_3_Fan_Status          2438 non-null   object        
 11  FC_Unit_3_Set_Point           

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from torch.utils.data import DataLoader, TensorDataset
import numpy as np

# Check for GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Input and categorical features
input_features = [
    'avg_temperature', 'avg_humidity', 'avg_co2',
    'weather_temp', 'weather_humid', 'total_energy',
    'total_power', 'total_current', 'hour', 'Day_of_week'
]
cat_features = ['weather_status']

# Fan unit columns
fan_unit_col = [col for col in merged_df.columns if "fc_unit" in col.lower()]

# Split X and y
X = merged_df[input_features + cat_features]
y = merged_df[fan_unit_col]

# Preprocessor for X
preprocessor_X = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), input_features),
        ('cat', OneHotEncoder(), cat_features)
    ]
)
X_processed = preprocessor_X.fit_transform(X)

# Preprocessor for y
cat_cols = [col for col in fan_unit_col if "Status" in col or "Mode" in col]
num_cols = [col for col in fan_unit_col if "Set_Point" in col]
preprocessor_y = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(), cat_cols),
        ('num', StandardScaler(), num_cols)
    ]
)
y_processed = preprocessor_y.fit_transform(y)



# Ensure y_processed is dense
if hasattr(y_processed, "toarray"):
    y_processed = y_processed.toarray()

# Reshape X and y for LSTM
X_processed = X_processed.reshape((X_processed.shape[0], 1, X_processed.shape[1]))
y_processed = y_processed.reshape((y_processed.shape[0], 1, y_processed.shape[1]))

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X_processed, y_processed, test_size=0.2, random_state=42)

# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32).to(device)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).to(device)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).to(device)

# Create DataLoader
def create_dataloader(X, y, batch_size=8):
    dataset = TensorDataset(X, y)
    return DataLoader(dataset, batch_size=batch_size, shuffle=True)

train_loader = create_dataloader(X_train_tensor, y_train_tensor)
test_loader = create_dataloader(X_test_tensor, y_test_tensor, batch_size=16)

# Define LSTM Model
class FanUnitLSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_categorical, output_numerical):
        super(FanUnitLSTM, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, batch_first=True)
        self.dropout = nn.Dropout(p=0.3)
        self.dense = nn.Linear(hidden_dim, hidden_dim)
        self.categorical_output = nn.Linear(hidden_dim, output_categorical)
        self.numerical_output = nn.Linear(hidden_dim, output_numerical)

    def forward(self, x):
        x, _ = self.lstm(x)
        x = self.dropout(x[:, -1, :])
        x = torch.relu(self.dense(x))
        categorical_out = torch.softmax(self.categorical_output(x), dim=1)
        numerical_out = self.numerical_output(x)
        return categorical_out, numerical_out

# Model parameters
input_dim = X_train_tensor.shape[2]
hidden_dim = 64
output_categorical = len(cat_cols)
output_numerical = len(num_cols)

# Instantiate the model
model = FanUnitLSTM(input_dim, hidden_dim, output_categorical, output_numerical).to(device)

# Loss and optimizer
categorical_loss_fn = nn.CrossEntropyLoss()
numerical_loss_fn = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

# Training loop
def train_model(model, train_loader, optimizer, epochs=100):
    model.train()
    for epoch in range(epochs):
        epoch_loss = 0
        for X_batch, y_batch in train_loader:
            optimizer.zero_grad()
            y_categorical = y_batch[:, :, :output_categorical].squeeze(1)
            y_numerical = y_batch[:, :, output_categorical:].squeeze(1)

            pred_categorical, pred_numerical = model(X_batch)

            loss_categorical = categorical_loss_fn(pred_categorical, y_categorical.argmax(dim=1))
            loss_numerical = numerical_loss_fn(pred_numerical, y_numerical)
            
            loss = loss_categorical + loss_numerical
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
        print(f"Epoch {epoch + 1}/{epochs}, Loss: {epoch_loss / len(train_loader):.4f}")

# Train the model
train_model(model, train_loader, optimizer, epochs=50)

# Save the model
torch.save(model.state_dict(), 'models/FanUnitLSTM.pth')

# Load and evaluate
model.load_state_dict(torch.load('models/FanUnitLSTM.pth'))
model.eval()


C:\Users\yy\anaconda3\envs\yy_1\Lib\site-packages\torch\nn\modules\loss.py:608: UserWarning: Using a target size (torch.Size([8, 56])) that is different to the input size (torch.Size([8, 8])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


RuntimeError: The size of tensor a (8) must match the size of tensor b (56) at non-singleton dimension 1